In [ ]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt

In [ ]:
calibration_image_files = glob.glob("../camera_cal/calibration*.jpg")

class Camera:
    def __init__(self, calibration_image_files):
        self.calibration_image_files = calibration_image_files
        
        self.objp = np.zeros((6 * 9, 3), np.float32);
        self.objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2);

        self.objpoints = []
        self.imgpoints = []
        
        self.mtx = None
        self.dist = None
        self.rvecs = None
        self.tvecs = None
        
    def calibrate(self):
        for idx, calibration_image_file in enumerate(self.calibration_image_files):
            
            img = cv2.imread(calibration_image_file)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)

            if ret == True:
                self.objpoints.append(self.objp)
                self.imgpoints.append(corners)
            
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(self.objpoints, self.imgpoints, img.shape[1::-1], None, None)
        
        self.mtx = mtx
        self.dist = dist
        self.rvecs = rvecs
        self.tvecs = tvecs
        
    def show_calibration(self):
        return self



camera = Camera(calibration_image_files)
camera.calibrate()
    
#myLine = Line("tests", camera, src, dst)
#print(myLine.img_file_name)

In [ ]:
test_image_files = glob.glob("../test_images/*.jpg");
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def processImage(image):

    result = line.processImage(image)

    return result

class Pipeline:
    def __init__(self, img_file_name, camera, src, dst):
        self.img_file_name = img_file_name
        self.camera = camera
        self.src = src
        self.dst = dst
        self.undistorted_img = None
        self.output_img = None

    def warpImage(self, img):
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        undistorted = cv2.undistort(img, self.camera.mtx, self.camera.dist, None, self.camera.mtx)
        
        img_size = (undistorted.shape[1], undistorted.shape[0])
        
        self.M = cv2.getPerspectiveTransform(self.src, self.dst)
        self.Minv = cv2.getPerspectiveTransform(self.dst, self.src)
        
        warped = cv2.warpPerspective(undistorted, self.M, img_size, flags=cv2.INTER_LINEAR)
        
        return warped
    
    def binaryFy(self, img):
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
        abs_sobelx = np.absolute(sobelx)
        scaled_sobel = np.uint8(255 * abs_sobelx / np.max(abs_sobelx))

        # Thresh
        thresh_min = 20
        thresh_max = 100
        sxbinary = np.zeros_like(scaled_sobel)
        sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1


        hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        s_channel = hls[:,:,2]
        l_channel = hls[:,:,1]

        s_thresh_min = 170
        s_thresh_max = 255
        s_binary = np.zeros_like(s_channel)
        s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1
        color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

        combined_binary = np.zeros_like(sxbinary)
        combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
        
        return combined_binary
    
    def findLaneLinesHist(self, img):
        bottom_half = img[img.shape[0] // 2:,:];
        histogram = np.sum(bottom_half, axis=0)
        #plt.plot(histogram)

        midpoint = np.int(histogram.shape[0]//2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint



        # HYPERPARAMETERS
        # Choose the number of sliding windows
        nwindows = 9
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50

        window_height = np.int(img.shape[0] // nwindows)

        leftx_current = leftx_base
        rightx_current = rightx_base

        out_img = np.dstack((img * 255, img * 255, img * 255))



        nonzero = img.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])

        left_lane_inds = []
        right_lane_inds = []

        for window in range(nwindows):

            win_y_low = img.shape[0] - (window + 1) * window_height
            win_y_high = img.shape[0] - window * window_height

            win_xleft_low = leftx_current - margin # Update this
            win_xleft_high = leftx_current + margin # Update this
            win_xright_low = rightx_current - margin # Update this
            win_xright_high = rightx_current + margin # Update this

            cv2.rectangle(out_img,(win_xleft_low,win_y_low),
                (win_xleft_high,win_y_high),(0,255,0), 2)
            cv2.rectangle(out_img,(win_xright_low,win_y_low),
                (win_xright_high,win_y_high),(0,255,0), 2)

            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
            (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0];
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
            (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0];

            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)

            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]

        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        self.left_fit = left_fit
        self.right_fit = right_fit

        ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
        self.ploty = ploty

        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        self.left_fitx = left_fitx
        self.right_fitx = right_fitx

        out_img[lefty, leftx] = [255, 0, 0]
        out_img[righty, rightx] = [0, 0, 255]

        #plt.plot(left_fitx, ploty, color='yellow')
        #plt.plot(right_fitx, ploty, color='yellow')

        #plt.imshow(out_img)
        #print(leftx_base, rightx_base)
        return out_img
        
    def calculateCurvature(self):
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension

        left_fit_cr = self.left_fit
        right_fit_cr = self.right_fit
        #y_eval = np.max(720)
        y_eval = 720

        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        
        self.left_curverad = left_curverad
        self.right_curverad = right_curverad
                
        #print(left_curverad, right_curverad)
        
    def drawLines(self, undistorted_img, combined_binary):
        # Create an image to draw the lines on
        warp_zero = np.zeros_like(undistorted_img).astype(np.uint8)

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([self.left_fitx, self.ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([self.right_fitx, self.ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(warp_zero, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(warp_zero, self.Minv, (combined_binary.shape[1], combined_binary.shape[0])) 
        # Combine the result with the original image
        result = cv2.addWeighted(undistorted_img, 1, newwarp, 0.3, 0)
        
        return result

    def processVideo(self):
        white_output = '../output_videos/solidWhiteRight.mp4'
        
        clip1 = VideoFileClip("../project_video.mp4")
        white_clip = clip1.fl_image(processImage) #NOTE: this function expects color images!!
        %time white_clip.write_videofile(white_output, audio=False)
        
        HTML("""
        <video width="960" height="540" controls>
          <source src="{0}">
        </video>
        """.format(white_output))
        
    def processImage2(self):
        img = cv2.imread(self.img_file_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        return self.processImage(img)
        
    def processImage(self, img):
        
        warpedImage = self.warpImage(img)
        
        binaryImage = self.binaryFy(warpedImage)
        
        processed_img = self.findLaneLinesHist(binaryImage)
        
        self.calculateCurvature()
        
        drawn_lines = self.drawLines(img, binaryImage)
        
        lc_str = "LC: " + str(int(self.left_curverad))
        rc_str = "RC: " + str(int(self.right_curverad))
        
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(drawn_lines,lc_str,(img.shape[1] // 2 -300, img.shape[0] // 2), font, 2, (255, 0, 0), 4, cv2.LINE_AA)
        cv2.putText(drawn_lines,rc_str,(img.shape[1] // 2 +100, img.shape[0] // 2), font, 2, (255, 0, 0), 4, cv2.LINE_AA)
        
        plt.imshow(drawn_lines)
        plt.show()
        
        return drawn_lines
        

src = np.float32(
[[715,  450],
 [1270, 700],
 [200,  700],
 [590,  450]])
    
dst = np.float32(
    [[1200, 100],
     [1200, 700],
     [200, 700],
     [200, 100]])

line = Pipeline(test_image_files[1], camera, src, dst)
line.processImage2()
line.processVideo()